## demo for learning embedding learning
Run under python3.6, numpy1.16.1

This demo is based 5 times reps.

In [1]:
import numpy as np
import funs
from scipy import sparse
from sklearn.svm import LinearSVC
from sklearn.metrics import hinge_loss

In [2]:
np.random.seed(21)
P2P_data = funs.P_data()
P2P_data.load_data('wiki.mat')
p, d = 5, P2P_data.weight.shape[0]

In [3]:
# LLE
LLE = funs.GEC(p=p)
P = sparse.eye(d) - P2P_data.weight
P = np.dot(P.T, P)
P = P.toarray()
eigvalue, eigvector = np.linalg.eig(P)
eigen_index = np.argsort(eigvalue)
LLE.X = eigvector[:,eigen_index[1:p+1]].real

In [5]:
## Laplacian eigenmaps
from sklearn.manifold import spectral_embedding
from scipy.sparse import csgraph
LE = funs.GEC(p=p)
LE.X = spectral_embedding(P2P_data.weight, n_components=p)

C_range = [5.*1e-5, 1e-4, 5.*1e-4, 1e-3, 5.*1e-3]
delta_range = [1e-4]
C1_range = 10**np.arange(-3, 3., .3)

/home/ben/pyenv/EL/lib/python3.6/site-packages/sklearn/utils/validation.py:72: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  return f(**kwargs)


In [6]:
echo_perf, LLE_perf, LE_perf = [], [], []
for r in range(5):
	train, valid, test = P2P_data.split_data()
	echo_cv, LLE_cv, LE_cv = [], [], []
	# for GEC method
	for delta in delta_range:
		echo = funs.GEC(p=p)
		echo.weight_inv(data=train)
		weight_inv_mat = echo.weight_inv_mat
		for C1 in C_range:
			for C2 in C_range:
				echo = funs.GEC(p=p)
				echo.weight_inv_mat = weight_inv_mat
				echo.fit(data=train, C1=C1, C2=C2, delta=delta)
				mce_echo_train = echo.perf(data=train)
				mce_echo_valid = echo.perf(data=valid)
				mce_echo_test = echo.perf(data=test)
				print('echo with C1: %.3f, C2: %.3f, delta: %.3f, train_perf: %.3f, valid_perf: %.3f, test_perf: %.3f' %(C1, C2, delta, mce_echo_train, mce_echo_valid, mce_echo_test))
				echo_cv.append([C1, C2, delta, mce_echo_train, mce_echo_valid, mce_echo_test])
	echo_cv = np.array(echo_cv)
	print(echo_cv[np.argmin(echo_cv[:,-2])])
	echo_perf.append(echo_cv[np.argmin(echo_cv[:,-2]), -1])

	## learning for LLE
	for C in C1_range:
		learner_LLE = LinearSVC(random_state=0, C=C, loss='hinge', max_iter=10000, fit_intercept=False)
		learner_LLE.fit(X=LLE.X[train.id], y=train.y)
		LLE.beta = learner_LLE.coef_[0]
		mce_LLE_valid = LLE.perf(data=valid)
		mce_LLE_test = LLE.perf(data=test)
		LLE_cv.append([mce_LLE_valid, mce_LLE_test])
	LLE_cv = np.array(LLE_cv)
	LLE_perf.append(LLE_cv[np.argmin(LLE_cv[:,-2]), -1])

	## Laplacian eigenmaps
	for C in C1_range:
		learner_LE = LinearSVC(random_state=0, C=C, class_weight='balanced', loss='hinge', max_iter=10000, fit_intercept=False)
		learner_LE.fit(X=LE.X[train.id], y=train.y)
		LE.beta = learner_LE.coef_[0]
		mce_LE_train = LE.perf(data=train)
		mce_LE_valid = LE.perf(data=valid)
		mce_LE_test = LE.perf(data=test)
		LE_cv.append([mce_LE_valid, mce_LE_test])
	LE_cv = np.array(LE_cv)
	LE_perf.append(LE_cv[np.argmin(LE_cv[:,-2]), -1])

Fit GEC: iteration: 0, obj: 2673.734, diff: 2672.734
- embedding block updated with obj: 2386.119
- learning block updated with obj: 2380.988
Fit GEC: iteration: 1, obj: 2380.988, diff: 0.109
- embedding block updated with obj: 2387.426
- learning block updated with obj: 2385.860
Fit GEC: iteration: 2, obj: 2385.860, diff: 0.002
- embedding block updated with obj: 2387.825
- learning block updated with obj: 2387.347
Fit GEC: iteration: 3, obj: 2387.347, diff: 0.001
echo with C1: 0.000, C2: 0.000, delta: 0.000, train_perf: 0.196, valid_perf: 0.194, test_perf: 0.199
Fit GEC: iteration: 0, obj: 2600.266, diff: 2599.266
- embedding block updated with obj: 2363.723
- learning block updated with obj: 2363.490
Fit GEC: iteration: 1, obj: 2363.490, diff: 0.091
- embedding block updated with obj: 2358.366
- learning block updated with obj: 2358.081
Fit GEC: iteration: 2, obj: 2358.081, diff: 0.002
- embedding block updated with obj: 2351.826
- learning block updated with obj: 2351.479
Fit GEC: 

Fit GEC: iteration: 0, obj: 2591.415, diff: 2590.415
- embedding block updated with obj: 2239.666
- learning block updated with obj: 2173.764
Fit GEC: iteration: 1, obj: 2173.764, diff: 0.161
- embedding block updated with obj: 1663.726
- learning block updated with obj: 1459.609
Fit GEC: iteration: 2, obj: 1459.609, diff: 0.329
- embedding block updated with obj: 1450.601
- learning block updated with obj: 1425.086
Fit GEC: iteration: 3, obj: 1425.086, diff: 0.024
- embedding block updated with obj: 1415.928
- learning block updated with obj: 1393.231
Fit GEC: iteration: 4, obj: 1393.231, diff: 0.022
- embedding block updated with obj: 1385.447
- learning block updated with obj: 1363.491
Fit GEC: iteration: 5, obj: 1363.491, diff: 0.021
- embedding block updated with obj: 1358.446
- learning block updated with obj: 1337.947
Fit GEC: iteration: 6, obj: 1337.947, diff: 0.019
- embedding block updated with obj: 1334.259
- learning block updated with obj: 1317.981
Fit GEC: iteration: 7, o

- embedding block updated with obj: 1271.182
- learning block updated with obj: 1271.176
Fit GEC: iteration: 12, obj: 1271.176, diff: 0.000
echo with C1: 0.001, C2: 0.000, delta: 0.000, train_perf: 0.196, valid_perf: 0.194, test_perf: 0.199
Fit GEC: iteration: 0, obj: 2151.697, diff: 2150.697
- embedding block updated with obj: 1062.925
- learning block updated with obj: 1056.342
Fit GEC: iteration: 1, obj: 1056.342, diff: 0.509
- embedding block updated with obj: 1067.574
- learning block updated with obj: 1060.894
Fit GEC: iteration: 2, obj: 1060.894, diff: 0.004
- embedding block updated with obj: 1072.646
- learning block updated with obj: 1065.871
Fit GEC: iteration: 3, obj: 1065.871, diff: 0.005
- embedding block updated with obj: 1078.144
- learning block updated with obj: 1071.316
Fit GEC: iteration: 4, obj: 1071.316, diff: 0.005
- embedding block updated with obj: 1084.150
- learning block updated with obj: 1077.276
Fit GEC: iteration: 5, obj: 1077.276, diff: 0.006
- embedding

Fit GEC: iteration: 0, obj: 1724.111, diff: 1723.111
- embedding block updated with obj: 1581.454
- learning block updated with obj: 1566.144
Fit GEC: iteration: 1, obj: 1566.144, diff: 0.092
- embedding block updated with obj: 1495.485
- learning block updated with obj: 1489.756
Fit GEC: iteration: 2, obj: 1489.756, diff: 0.049
- embedding block updated with obj: 1454.113
- learning block updated with obj: 1451.195
Fit GEC: iteration: 3, obj: 1451.195, diff: 0.026
- embedding block updated with obj: 1427.278
- learning block updated with obj: 1425.617
Fit GEC: iteration: 4, obj: 1425.617, diff: 0.018
- embedding block updated with obj: 1410.002
- learning block updated with obj: 1408.905
Fit GEC: iteration: 5, obj: 1408.905, diff: 0.012
- embedding block updated with obj: 1397.048
- learning block updated with obj: 1396.280
Fit GEC: iteration: 6, obj: 1396.280, diff: 0.009
- embedding block updated with obj: 1386.568
- learning block updated with obj: 1386.021
Fit GEC: iteration: 7, o

Fit GEC: iteration: 0, obj: 882.003, diff: 881.003
- embedding block updated with obj: 900.535
- learning block updated with obj: 895.763
Fit GEC: iteration: 1, obj: 895.763, diff: 0.016
- embedding block updated with obj: 912.730
- learning block updated with obj: 909.588
Fit GEC: iteration: 2, obj: 909.588, diff: 0.015
- embedding block updated with obj: 924.352
- learning block updated with obj: 922.312
Fit GEC: iteration: 3, obj: 922.312, diff: 0.014
- embedding block updated with obj: 935.234
- learning block updated with obj: 933.949
Fit GEC: iteration: 4, obj: 933.949, diff: 0.013
- embedding block updated with obj: 944.577
- learning block updated with obj: 943.782
Fit GEC: iteration: 5, obj: 943.782, diff: 0.011
- embedding block updated with obj: 952.807
- learning block updated with obj: 952.350
Fit GEC: iteration: 6, obj: 952.350, diff: 0.009
- embedding block updated with obj: 959.132
- learning block updated with obj: 958.856
Fit GEC: iteration: 7, obj: 958.856, diff: 0.0

...

- embedding block updated with obj: 1033.885
- learning block updated with obj: 1031.460
Fit GEC: iteration: 6, obj: 1031.460, diff: 0.003
- embedding block updated with obj: 1031.420
- learning block updated with obj: 1028.721
Fit GEC: iteration: 7, obj: 1028.721, diff: 0.003
- embedding block updated with obj: 1028.883
- learning block updated with obj: 1025.808
Fit GEC: iteration: 8, obj: 1025.808, diff: 0.003
- embedding block updated with obj: 1026.166
- learning block updated with obj: 1022.766
Fit GEC: iteration: 9, obj: 1022.766, diff: 0.003
- embedding block updated with obj: 1023.386
- learning block updated with obj: 1019.624
Fit GEC: iteration: 10, obj: 1019.624, diff: 0.003
- embedding block updated with obj: 1020.656
- learning block updated with obj: 1016.484
Fit GEC: iteration: 11, obj: 1016.484, diff: 0.003
- embedding block updated with obj: 1017.751
- learning block updated with obj: 1013.360
Fit GEC: iteration: 12, obj: 1013.360, diff: 0.003
- embedding block update

- embedding block updated with obj: 1332.013
- learning block updated with obj: 1327.780
Fit GEC: iteration: 6, obj: 1327.780, diff: 0.016
- embedding block updated with obj: 1313.250
- learning block updated with obj: 1309.472
Fit GEC: iteration: 7, obj: 1309.472, diff: 0.014
- embedding block updated with obj: 1295.704
- learning block updated with obj: 1292.204
Fit GEC: iteration: 8, obj: 1292.204, diff: 0.013
- embedding block updated with obj: 1279.202
- learning block updated with obj: 1275.922
Fit GEC: iteration: 9, obj: 1275.922, diff: 0.013
- embedding block updated with obj: 1263.869
- learning block updated with obj: 1260.670
Fit GEC: iteration: 10, obj: 1260.670, diff: 0.012
- embedding block updated with obj: 1249.487
- learning block updated with obj: 1246.226
Fit GEC: iteration: 11, obj: 1246.226, diff: 0.011
- embedding block updated with obj: 1236.007
- learning block updated with obj: 1232.602
Fit GEC: iteration: 12, obj: 1232.602, diff: 0.011
- embedding block update

- embedding block updated with obj: 1437.522
- learning block updated with obj: 1435.212
Fit GEC: iteration: 4, obj: 1435.212, diff: 0.017
- embedding block updated with obj: 1418.061
- learning block updated with obj: 1416.352
Fit GEC: iteration: 5, obj: 1416.352, diff: 0.013
- embedding block updated with obj: 1401.747
- learning block updated with obj: 1400.437
Fit GEC: iteration: 6, obj: 1400.437, diff: 0.011
- embedding block updated with obj: 1389.302
- learning block updated with obj: 1388.226
Fit GEC: iteration: 7, obj: 1388.226, diff: 0.009
- embedding block updated with obj: 1378.771
- learning block updated with obj: 1377.849
Fit GEC: iteration: 8, obj: 1377.849, diff: 0.007
- embedding block updated with obj: 1370.463
- learning block updated with obj: 1369.638
Fit GEC: iteration: 9, obj: 1369.638, diff: 0.006
- embedding block updated with obj: 1362.686
- learning block updated with obj: 1361.945
Fit GEC: iteration: 10, obj: 1361.945, diff: 0.006
- embedding block updated 

- embedding block updated with obj: 807.550
- learning block updated with obj: 806.527
Fit GEC: iteration: 3, obj: 806.527, diff: 0.022
- embedding block updated with obj: 792.962
- learning block updated with obj: 792.221
Fit GEC: iteration: 4, obj: 792.221, diff: 0.018
- embedding block updated with obj: 780.578
- learning block updated with obj: 780.046
Fit GEC: iteration: 5, obj: 780.046, diff: 0.015
- embedding block updated with obj: 770.003
- learning block updated with obj: 769.626
Fit GEC: iteration: 6, obj: 769.626, diff: 0.013
- embedding block updated with obj: 761.370
- learning block updated with obj: 761.098
Fit GEC: iteration: 7, obj: 761.098, diff: 0.011
- embedding block updated with obj: 754.147
- learning block updated with obj: 753.951
Fit GEC: iteration: 8, obj: 753.951, diff: 0.009
- embedding block updated with obj: 747.908
- learning block updated with obj: 747.768
Fit GEC: iteration: 9, obj: 747.768, diff: 0.008
- embedding block updated with obj: 742.530
- le

- embedding block updated with obj: 803.306
- learning block updated with obj: 802.369
Fit GEC: iteration: 7, obj: 802.369, diff: 0.020
- embedding block updated with obj: 789.661
- learning block updated with obj: 788.979
Fit GEC: iteration: 8, obj: 788.979, diff: 0.017
- embedding block updated with obj: 777.675
- learning block updated with obj: 777.189
Fit GEC: iteration: 9, obj: 777.189, diff: 0.015
- embedding block updated with obj: 767.365
- learning block updated with obj: 767.022
Fit GEC: iteration: 10, obj: 767.022, diff: 0.013
- embedding block updated with obj: 759.189
- learning block updated with obj: 758.941
Fit GEC: iteration: 11, obj: 758.941, diff: 0.011
- embedding block updated with obj: 752.079
- learning block updated with obj: 751.902
Fit GEC: iteration: 12, obj: 751.902, diff: 0.009
- embedding block updated with obj: 746.123
- learning block updated with obj: 745.998
Fit GEC: iteration: 13, obj: 745.998, diff: 0.008
- embedding block updated with obj: 741.172


In [7]:
echo_perf, LLE_perf, LE_perf = np.array(echo_perf), np.array(LLE_perf), np.array(LE_perf)
print("echo perfm: %.3f(%.3f)" %(echo_perf.mean(), echo_perf.std()/np.sqrt(5)))
print("LLE perfm: %.3f(%.3f)" %(LLE_perf.mean(), LLE_perf.std()/np.sqrt(5)))
print("LE perfm: %.3f(%.3f)" %(LE_perf.mean(), LE_perf.std()/np.sqrt(5)))

echo perfm: 0.199(0.003)
LLE perfm: 0.485(0.008)
LE perfm: 0.595(0.059)
